In [12]:
import time

import cv2
import itertools
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from pycocotools.coco import COCO
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image
from itertools import groupby
from skimage import io
from sympy.core.random import randint
from tqdm import tqdm

import pycocotools.mask as mask_util

In [13]:
def rle_decode(mask_rle, shape):
    """
    Decodes run-length encoded segmentation mask string into 2d array

    Parameters
    ----------
    :param rle_mask (str): Run-length encoded segmentation mask string.
    :param shape (tuple): (height, width) of array to return
    :return mask [numpy.ndarray of shape (height, width)]: Decoded 2d segmentation mask
    """
    # Splits the RLE string into a list of string by whitespaces.
    s = mask_rle.split()

    # This creates two numpy arrays, one with the RLE starts and one with their respective lengths
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]

    # To obtain the end point we need to substract 1 to the length or start because the initial point counts.
    starts -= 1
    ends = starts + lengths

    # Create a 1D array of size H*W of zeros
    mask = np.zeros(shape[0]*shape[1], dtype=np.uint8)

    # Fill this array with ones in the positions where there is a mask using the RLE information
    for start, end in zip(starts, ends):
        mask[start:end] = 1

    # Reshape the 1D array into a 2D array so we can finally get the binary 2D mask.
    mask = mask.reshape(shape)
    return mask.T

In [14]:
def binary_mask_to_rle(binary_mask):
    """
    Checkout: https://cocodataset.org/#format-results
    :param mask [numpy.ndarray of shape (height, width)]: Decoded 2d segmentation mask

    This function returns the following dictionary:
    {
        "counts": encoded mask suggested by the official COCO dataset webpage.
        "size": the size of the input mask/image
    }
    """
    # Create dictionary for the segmentation key in the COCO dataset
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    # We need to convert it to a Fortran array
    binary_mask_fortran = np.asfortranarray(binary_mask)
    # Encode the mask as specified by the official COCO format
    encoded_mask = mask_util.encode(binary_mask_fortran)
    # We must decode the byte encoded string or otherwise we cannot save it as a JSON file
    rle["counts"] = encoded_mask["counts"].decode()
    return rle

In [15]:
def create_coco_format_json(data_frame, classes, filepaths, split=0.7):
    """
    This function creates a COCO dataset.
    :param data_frame: pandas dataframe with an "id" column.
    :param classes: list of strings where each string is a class.
    :param filepaths: a list of strings containing all images paths
    :return dataset_coco_format: COCO dataset (JSON).
    """
    images_train = []
    images_val = []
    annotations_train = []
    annotations_val = []
    categories = []
    count_train = 0
    count_val = 0

    # Creates a categories list, i.e: [{'id': 0, 'name': 'a'}, {'id': 1, 'name': 'b'}, {'id': 2, 'name': 'c'}]
    for idx, class_ in enumerate(classes):
        categories.append(
            {
                "id": class_,
                "name": f'class_{class_}'
            }
        )

    # Iterate over image filepaths
    for file in tqdm(filepaths):
        train_sample = True if random.random() < split else False
        file = file[-15:]  #Get only the name
        cond = data_frame['ImageId'] == file
        index_list = data_frame.index[cond].tolist()
        # Get the image id, e.g: "10044"
        #print(filepath)
        file_id = file
        #print(file_id)
        # Get the image height, e.g: 360 (px)
        height = 256
        # Get the image width, e.g: 310 (px)
        width = 256
        # One image has many annotations associated to it (1 for each class), get a list with the indices.
        #print(ids)
        # Get filename
        file_name = file_id
        # Adding images which have annotations
        img = {
                "id": file_id,
                "width": width,
                "height": height,
                "file_name": file_name}
        if train_sample: 
            images_train.append(img) 
        else:
            images_val.append(img)
        
        if len(index_list) == 0: # there is no defect on the image
            mk = np.zeros((256,256), dtype=np.uint8) # create blank mask
            enc = binary_mask_to_rle(mk)
            seg = {
                'segmentation': enc,
                'bbox': [None, None, None, None],
                'area': None,
                'image_id':file_id,
                'category_id': 5,
                'iscrowd':0,
                'id': count_train if train_sample else count_val
            }
            if train_sample:
                annotations_train.append(seg)
                count_train +=1
            else:
                annotations_val.append(seg)
                count_val +=1
        else:
            for idx in index_list:
                mk = rle_decode(data_frame.iloc[idx]['EncodedPixels'], (256, 256))
                ys, xs = np.where(mk)
                x1, x2 = min(xs), max(xs)
                y1, y2 = min(ys), max(ys)
                enc = binary_mask_to_rle(mk)
                seg = {
                    'segmentation': enc,
                    'bbox': [int(x1), int(y1), int(x2-x1+1), int(y2-y1+1)],
                    'area': int(np.sum(mk)),
                    'image_id':file_id,
                    'category_id':int(data_frame.iloc[idx]['ClassId']),
                    'iscrowd':0,
                    'id': count_train if train_sample else count_val
                }
                if train_sample:
                    annotations_train.append(seg)
                    count_train +=1
                else:
                    annotations_val.append(seg)
                    count_val +=1


    # Create the dataset
    dataset_train_coco_format = {
        "categories": categories,
        "images": images_train,
        "annotations": annotations_train,
    }
    dataset_val_coco_format = {
        "categories": categories,
        "images": images_val,
        "annotations": annotations_val,
    }

    return dataset_train_coco_format, dataset_val_coco_format


def sep():
    print("-"*100)

In [16]:
height = 256
width = 256
data_path = '/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/train_subimages'

In [17]:
df = pd.read_csv('/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/subimages.csv')
#Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

In [18]:
classes = sorted(df["ClassId"].unique().tolist())
classes.append(5)  # Class for images without any defect

In [19]:
filepaths = list()

for (dirpath, dirnames, filenames) in os.walk(data_path):
    filepaths += [os.path.join(dirpath, file) for file in filenames if file.endswith(".png")]

In [20]:
g_json_train, g_json_val = create_coco_format_json(df, classes, filepaths, 0.8)

100%|██████████| 87976/87976 [02:24<00:00, 607.91it/s]


In [21]:
with open('/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/annotations_COCO_train.json', 'w', encoding='utf-8') as f:
    json.dump(g_json_train, f, indent=4)

In [22]:
with open('/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/annotations_COCO_val.json', 'w', encoding='utf-8') as f:
    json.dump(g_json_val, f, indent=4)